#Workshop I - System Analysis

1. First, a function to generate a sequence based on typical for DNA nucleotid bases is created. Here the idea is to use random numbers to get a random sequence of a random size (between 10 and 20.)

In [ ]:
import random


def create_sequence() -> str:
    """
    This function is used to generate a random genetic sequence.

    Returns:
    - str: random genetic sequence
    """
    nucleotid_bases = ["A", "C", "G", "T"]
    size_sequence = random.randint(10, 20)
    new_sequence = [nucleotid_bases[random.randint(0, 3)] for i in range(size_sequence)]
    return "".join(new_sequence)

2. Based on create_sequences() function, the idea is to generate a dataset of genetic sequences, just persisting all sequences into a list.

In [ ]:
def create_dataset(dataset_size: int) -> list:
    """
    This function is used to create a dataset composed by a set of genetic sequences.

    Parameters:
    - dataset_size (int): size of the dummy dataset to be generated.

    Returns:
    - list: a list of genetic sequences
    """
    dataset = [create_sequence() for i in range(dataset_size)]
    return dataset

3. As the goal is to get the motif in the sequence, first task for that purpose is to generate all possible combinations of secuences of an specific size.

In [ ]:
def get_combinations(n: int, sequences: list, bases: list) -> list:
    """
    This method is used to generate a set of combinations based on a list of nucleotid bases.
    To make easy the process, this function is defined as a recurssion.

    Parameters:
    - n (int): amount of elements of each combination
    - sequences (list): list of recursive sequences obtained
    - bases (list): list of nucleotid bases to be used

    Returns:
    - list: list of combinations
    """
    if n == 1:
        return [sequence + base for sequence in sequences for base in bases]
    else:
        sequence_ = [sequence + base for sequence in sequences for base in bases]
        return get_combinations(n - 1, sequence_, bases)

4. Each motif candidate should be verified into each sequence in the dataset, just to count the motif occurences.

In [ ]:
def count_motif(motif: str, sequences_dataset: list) -> int:
    """
    This function is used to count the number of times a motif appears in a set of genetic sequences.

    Parameters:
    - motif (str): genetic motif to be searched.
    - sequences_dataset (list): list of genetic sequences.

    Returns:
    - int: number of times the motif appears in the dataset.
    """
    count = 0
    for sequence in sequences_dataset:
        count += sequence.count(motif)
    return count

5. Now it is time to get all motif candidates (combinations of a specific size), count occurrences for each motif, and choose as the motif winner the only one who has more ocurreces.

In [ ]:
from typing import Union
def get_motif(motif_size: int, sequences_dataset: list) -> Union[str, int]:
    """
    This function is used to get the motif with the highest count in a set of genetic sequences.

    Parameters:
    - motif_size (int): size of the motif to be searched.
    - sequences_db (list): list of genetic sequences.

    Returns:
    - (str, int): motif with the highest count and the number of times it appears in the dataset.
    """
    nucleotid_bases = ["A", "C", "G", "T"]
    combinations = get_combinations(motif_size, [""], nucleotid_bases)
    # get motif with the highest count
    max_counter = 0
    motif_winner = ""
    for motif_candidate in combinations:
        temp_counter = count_motif(motif_candidate, sequences_dataset)
        if temp_counter > max_counter:
            max_counter = temp_counter
            motif_winner = motif_candidate

    return motif_winner, max_counter

6. Now, just a loop of callings looking for some different motifs, in order you have a better idea about possible motifs and occurences, so you could make a better analysis. Also, motifs of size 6 and 8 will be generated.

In [ ]:
for size in [6, 8]:
    print(f"\nMotifs of size: {size}")
    for i in range(3):
        print(get_motif(size, create_dataset(50000)))


Motifs of size: 6
('ACGGGT', 3)
('ATGACT', 3)
('AACCAA', 2)
('ACGCGC', 5)
('CTCAGC', 3)
('ACAATA', 3)
('AACAAC', 3)
('ACACCC', 2)
('CTATCT', 3)
('CGTCCC', 3)

Motifs of size: 8
('CCGTCCGT', 2)
('ACCCATGA', 2)
('AAACGAGA', 1)
('AACTAATA', 2)
('CTTCTCCT', 2)
('TGCTGTAA', 2)
('GAAAGCCG', 2)
('GTGGCCAG', 2)
('CAAACTAT', 2)
('CGACAGTC', 2)


7. Now, as the idea is to define how Shannon Entropy could help us to filter maybe not relevant sequences in order to get more relevant motifs. You could check equation here.

In [ ]:
import math


def calculate_shannon_entrophy(sequence: str) -> float:
    """
    This function is used to calculate the Shannon Entropy of a genetic sequence.

    Parameters:
    - sequence (str): genetic sequence.

    Returns:
    - float: Shannon Entropy of the sequence.
    """
    counts = {}
    total = 0
    for base in sequence:
        counts[base] = counts.get(base, 0) + 1
        total += 1

    entrophy = 0
    for count in counts.values():
        probability = count / total
        entrophy -= probability * math.log2(probability)

    return entrophy

8. As a Shannon Entropy Calculation function is created, it is possible to make a dataset filter before to check for motifs.

In [ ]:
def filter_shannon(sequence: str, threshold: float) -> bool:
    """
    This function is used to filter genetic sequences based on their Shannon Entropy.

    Parameters:
    - sequence (str): genetic sequence.
    - threshold (float): the threshold value for entropy filtering.

    Returns:
    - bool: True if the sequence passes the filter, False otherwise.
    """
    entrophy = calculate_shannon_entrophy(sequence)
    return entrophy >= threshold

9. Last part of the workshop, it is just filter sequences in the full motif search process.

In [ ]:
entropy_threshold = 0.5
for size in [6, 8]:
    print(f"\nAfter filter, motifs of size: {size}")
    for i in range(10):
        dataset = create_dataset(50000)
        dataset = [seq for seq in dataset if filter_shannon(seq, entropy_threshold)]
        print(f"Dataset size: {len(dataset)}, Motif: {get_motif(size, dataset)}")


After filter, motifs of size: 6
Dataset size: 50000, Motif: ('GCCCCA', 164)
Dataset size: 50000, Motif: ('GTTGGC', 162)
Dataset size: 49998, Motif: ('CTGGTT', 161)
Dataset size: 49999, Motif: ('AGCGGG', 163)
Dataset size: 49998, Motif: ('CACTGG', 166)
Dataset size: 50000, Motif: ('CCAGAT', 169)
Dataset size: 49998, Motif: ('GCGTAA', 159)
Dataset size: 50000, Motif: ('GGTAGT', 160)
Dataset size: 49999, Motif: ('ACAGAT', 168)
Dataset size: 49999, Motif: ('GCTAGA', 174)

After filter, motifs of size: 8
Dataset size: 49998, Motif: ('ACTGAAAG', 22)
Dataset size: 49998, Motif: ('GGGCTGAG', 21)
Dataset size: 49999, Motif: ('GAATGCCA', 20)
Dataset size: 50000, Motif: ('GGTAAGAA', 20)
Dataset size: 49999, Motif: ('TTCGCACC', 20)
Dataset size: 50000, Motif: ('GCTTGGGC', 19)
Dataset size: 49999, Motif: ('GTCCGCGA', 21)
Dataset size: 49999, Motif: ('CATCAGGT', 19)
Dataset size: 49999, Motif: ('TAGTCAAT', 20)
Dataset size: 50000, Motif: ('ACTTTGTT', 21)
